In [25]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import seaborn as sns
from sklearn import preprocessing
from datetime import datetime
import plotly.express as px
import statistics

In [2]:
dt=pd.read_csv('Data/packets.csv')

In [3]:
print(dt.head())

            Source      Destination Protocol  Length        DT  source port  \
0   142.250.180.42  192.168.137.185      UDP      74  0.000000        443.0   
1   142.250.180.42  192.168.137.185      UDP      74  0.063438        443.0   
2   142.250.180.42  192.168.137.185      UDP      74  0.123586        443.0   
3  192.168.137.185   142.250.180.42      UDP      75  0.024475      62030.0   
4  192.168.137.185   142.250.180.42      UDP      77  0.000000      62030.0   

     Absolute Time  Cumulative Bytes  dest port  
0  16:47:47.942724                74    62030.0  
1  16:47:48.006162               148    62030.0  
2  16:47:48.129748               222    62030.0  
3  16:47:48.154223               297      443.0  
4  16:47:48.154223               374      443.0  


In [4]:
myip='192.168.137.185'
direction=[]


In [5]:
for i in dt['Destination']:
    if(i==myip):
        direction.append('in')
    else:
        direction.append('out')

In [6]:
dt.insert(0,'direction',direction)

In [7]:
print(dt.isnull().sum()) 

direction             0
Source                0
Destination           0
Protocol              0
Length                0
DT                    0
source port         145
Absolute Time         0
Cumulative Bytes      0
dest port           145
dtype: int64


In [8]:
dt['source port'].fillna(443,inplace=True)

In [112]:
inp=[]
out=[]
leng_mean=[]
leng_std=[]
leng_min=[]
leng_max=[]
sum_dt=[]
port=[]

In [113]:
start=0
end=100
inps=0
outs=0
cc=round(dt.shape[0]/100)
tempdf=[]
tempport=[]
tempdt=0
for i in range(cc):
    for k in range(start,end):
        if(dt['direction'][k]=='in'):
            inps=inps+1
        else:
            outs=outs+1
        tempdf.append(dt['Length'][k])
        tempdt=tempdt+dt['DT'][k]
        tempport.append(dt['source port'][k])

        
    leng_mean.append(statistics.mean(tempdf))
    leng_std.append(statistics.stdev(tempdf))
    leng_min.append(min(tempdf))
    leng_max.append(max(tempdf))
    sum_dt.append(tempdt)
    if(max(tempport,key=tempport.count)==443):
        port.append('stream')
    else:
        port.append('not stream')
    inp.append(inps)
    out.append(outs)
    start=start+100
    end=end+100
    inps=0
    outs=0
    tempdf=[]
    tempport=[]
    tempdt=0


In [114]:
df=pd.DataFrame()

In [115]:
df.insert(0,'inp',inp)
df.insert(1,'out',out)
df.insert(2,'leng_mean',leng_mean)
df.insert(3,'leng_std',leng_std)
df.insert(4,'leng_min',leng_min)
df.insert(5,'leng_max',leng_max)
df.insert(6,'sum_dt',sum_dt)
df.insert(7,'port',port)


In [117]:
df.to_csv('Data/packets_stat.csv')

In [116]:
fig=px.histogram(df,x='port',y='inp',color='port',nbins=20)
fig.show()